# Run the pipeline from the notebook


- work with Weakly_2022_34
- use jupyter kernel LSST

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2022/09/20


In [5]:
! eups list -s | grep LOCAL

atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2022_34/atmospec 	setup
drp_pipe              LOCAL:/home/d/dagoret/repos/repos_w_2022_34/drp_pipe 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-4.1.0/eups 	setup


In [1]:
# Basic imports
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

from pprint import pprint

In [2]:
from lsst.ctrl.mpexec import SimplePipelineExecutor
from lsst.pipe.base import Pipeline

In [3]:
from lsst.daf.butler import Butler

repo = "/sdf/group/rubin/repo/main"

butler = Butler(repo)
registry = butler.registry

### DRP pipeline for Latiss
https://github.com/lsst/drp_pipe/tree/main/pipelines/LATISS

In [4]:
# set up an output collection with your username
#outputCollection = "u/nate2/analysisToolsExample"
inputCollections = ["LATISS/raw/all","refcats","LATISS/calib"]
outputCollection = "u/dagoret/test_latiss_drp"

# this can be skipped if you already have a read writable butler setup (above is read only)
butlerRW = SimplePipelineExecutor.prep_butler(repo, inputs = inputCollections,output=outputCollection)

# load in the pipeline to run
#pipeline = Pipeline.from_uri("$ANALYSIS_TOOLS_DIR/pipelines/coaddQualityCore.yaml")


pipeline = Pipeline.from_uri("$DRP_PIPE_DIR/pipelines/LATISS/DRP.yaml")

In [6]:
# override a configuration within a certain AnalysisTool
#configKey = "plots.shapeSizeFractionalDiffScatter.prep.selectors.snSelector.threshold"
#pipeline.addConfigOverride("analyzeObjectTableCore", configKey, 400)

#bands = ['g', 'r', 'i', 'z']
#pipeline.addConfigOverride("analyzeObjectTableCore", "bands", bands)
#pipeline.addConfigOverride("catalogMatchTract", "bands", bands)
#pipeline.addConfigOverride("refCatObjectTract", "bands", bands)

# restrict processing to the same dataId used above
#whereString = "tract = 9813 AND skymap = 'hsc_rings_v1'"
whereString = "exposure.day_obs={} and exposure.seq_num={} and instrument='LATISS'".format("20220607","195") 

# Prevent the executor from dumping plots into the notebook
backend_ =  mpl.get_backend() 
mpl.use("Agg")

executor = SimplePipelineExecutor.from_pipeline(pipeline, where=whereString, butler=butlerRW)
quanta = executor.run(True)

# Restore the ability for plots to be put into the notebook
mpl.use(backend_)

# If you only want to run one plot in a task in the pipeline do the following prior to execution
# pipeline.addConfigOverride("analyzeObjectTableCore", "plots", None)
# pipeline.addConfigOverride("analyzeObjectTableCore", "plots", ShapeSizeFractionalDiffScatterPlot)

/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/pipe_tasks/gf10b05e212+a1470f6de0/python/lsst/pipe/tasks/calibrate.py:330: FutureWarning: Config field astromRefObjLoader.ref_dataset_name is deprecated: This field is not used. It will be removed after v25.
  self.astromRefObjLoader.ref_dataset_name = "gaia_dr2_20200414"
/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/pipe_tasks/gf10b05e212+a1470f6de0/python/lsst/pipe/tasks/calibrate.py:331: FutureWarning: Config field photoRefObjLoader.ref_dataset_name is deprecated: This field is not used. It will be removed after v25.
  self.photoRefObjLoader.ref_dataset_name = "ps1_pv3_3pi_20170110"
/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/obs_lsst/g61e05cf539+a8371b4bcb/config/latiss/calibrate.py:6: FutureWarning: Config field photoRefObjLoader.ref_dataset_name is deprecated: This field is not used. It will be removed after v25.
  config.photoRefObjLoader.ref_dataset_name = PHOTO_REFCAT

Overriding default configuration file with /opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/dustmaps_cachedata/g41a3ec361e+c9b10be330/config/.dustmapsrc


FileNotFoundError: Not enough datasets (0) found for non-optional connection isr.flat (flat) with minimum=1 for quantum data ID {instrument: 'LATISS', detector: 0, exposure: 2022060700195, ...}.

In [16]:
datasetRefs = registry.queryDatasets(datasetType='postISRCCD', collections = outputCollection , where= "instrument='LATISS'")

MissingCollectionError: No collection with name 'u/dagoret/test_processstar01' found.